# Logit bias

From the OpenAI API, [logit bias](https://platform.openai.com/docs/api-reference/chat/create#chat/create-logit_bias) is taken as a mapping of tokens to their bias levels ranging from -100, a potential complete ban of the token, to 100, a potenital guarantee of selection. Lesser magnitude values like -1 or 1 will configure the possibility for the token's "selection": the chance it will appear in the output. 

The representative tokens for certain characters can be obtained from [OpenAI's tokenizer](https://platform.openai.com/tokenizer) or through other helpful programs. 

Tokenizers are sensitive to capitalization, spaces, and punctuation; it is crucial to consider all possible tokens of certain strings if logit bias is being implemented manually.

> A helpful tokenizer package to is `tiktoken`, check out the project [here](https://github.com/openai/tiktoken).

As a team of three, we worked on implementing logit bias on the Python branch by adding a dictionary parameter to `ChatRequestSettings` and `CompleteRequestSettings` and including them in the JSON requests sent to the OpenAI API.

See our [merged pull request #1880](https://github.com/microsoft/semantic-kernel/pull/1880) for the source code.

---

**Case 1: `ChefBot`**

In [1]:
# Logit bias is part of the offical Python module
!python -m pip -q install semantic-kernel

Imagine you are developing a `ChefBot` program that comes up with delicious recipes for a user while also keeping preferences and allergies in mind. Yum!

A user may ask:

```bash
User:> I want to make a smoothie, pls remember I am allergic to bananas!
```

Gathering the tokens for "bananas" yields `[3820, 15991]`, which should be banned from the outputted recipe. 

> As discussed earlier, however, to completely ban the tokens would mean having different variations of the string `"bananas"`.
> I will add these myself, but don't be discouraged if you aren't getting total bans at first!

Let's set up the `ChefBot` chat experience:

In [2]:
# Required imports

import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.connectors.ai.chat_request_settings import ChatRequestSettings

In [3]:
# Setup a chat service

# Note: you can download this notebook and run in a directory containing a .env with your OpenAI API key,
# or add it manually where the "key" string is. Be careful with your secrets!
USE_ENV = True
if USE_ENV:
    api_key, org_id = sk.openai_settings_from_dot_env()
else:
    api_key, org_id = "key", None

# Register the chat service into the kernel
openai_chat_completion = OpenAIChatCompletion("gpt-3.5-turbo", api_key, org_id)

# Ensuring it was instantiated correctly
assert openai_chat_completion


> For more complex use with the kernel and as a semantic function, you can also register it as a skill:
> ```python
> kernel = sk.Kernel()
> prompt_config = sk.PromptTemplateConfig.from_completion_parameters(max_tokens=2000, temperature=0.7, top_p=0.8)
> prompt_template = sk.ChatPromptTemplate("{{$user_input}}", kernel.prompt_template_engine, prompt_config)
> prompt_template.add_system_message("You are a chat bot specializing the culinary arts.")
> function_config = sk.SemanticFunctionConfig(prompt_config, prompt_template)
> chat_with_chef = kernel.register_semantic_function("ChefBot", "Chat", function_config)
> ```

The tokens must be set with a logit bias of -100 for a total ban. This is done within the `ChatRequestSettings` instance send with the request.

> `CompleteRequestSettings` is used in text completion requests.

In [4]:
settings = ChatRequestSettings(max_tokens=512, temperature=0.5, top_p=0.8)

tokens_to_ban = [3820, 15991, 2271, 35484, 25996, 3958]
for token in tokens_to_ban:
    settings.token_selection_biases[token] = -100       # no monkey business

Set up chat history and add the user's request.
> In an actual program, this can be done more succinctly using `input()` and some parsing.

In [5]:
user_request = "I want to make a smoothie, pls remember I am allergic to bananas!"

chat_messages = list()
chat_messages.append(("user", user_request))

User request in one hand, loaded settings in the other, let's get a response!

In [6]:
answer = await openai_chat_completion.complete_chat_async(chat_messages, settings)
chat_messages.append(("assistant", str(answer)))

Define a helper loop for printing out chat history:

In [7]:
def print_chat(msg_list):
    for role, msg in msg_list:
        if role == "user":
            print(f"User:> {msg}\n")
        else:
            print(f"ChefBot:> {msg}\n")

print_chat(chat_messages)

User:> I want to make a smoothie, pls remember I am allergic to bananas!

ChefBot:> No worries! Here's a delicious smoothie recipe that doesn't include bananas:

Ingredients:
- 1 cup of frozen berries (strawberries, blueberries, raspberries, or a mix)
- 1 cup of spinach or kale
- 1 cup of almond milk (or any other non-dairy milk of your choice)
- 1 tablespoon of honey or maple syrup (optional, for added sweetness)
- 1 tablespoon of chia seeds (optional, for added nutrition)
- Ice cubes (optional, for a colder smoothie)

Instructions:
1. Add the frozen berries, spinach or kale, almond milk, honey or maple syrup, and chia seeds to a blender.
2. Blend on high speed until smooth and creamy. If the mixture is too thick, you can add a little more almond milk to reach your desired consistency.
3. Taste the smoothie and adjust the sweetness if needed by adding more honey or maple syrup.
4. If you prefer a colder smoothie, add a few ice cubes and blend again until well combined.
5. Pour the smo

Hmm... other than the friendly assertion that the recipe does not include bananas, the response was pretty banana-less!

But how can you be certain it did not response as such only *because* you said you were allergic to bananas? 

Confirm by removing it from the prompt and restarting the chat history!

In [20]:
user_request = "I want to make a smoothie :)"

chat_messages = list()
chat_messages.append(("user", user_request))
answer = await openai_chat_completion.complete_chat_async(chat_messages, settings)
chat_messages.append(("assistant", str(answer)))

In [21]:
print_chat(chat_messages)

User:> I want to make a smoothie :)

ChefBot:> Great! Making a smoothie is a fun and healthy way to enjoy a refreshing drink. Here's a simple recipe to get you started:

Ingredients:
- 1 cup of your favorite fruits (such as bananas, berries, mango, or pineapple)
- 1 cup of liquid (such as milk, almond milk, coconut water, or yogurt)
- 1 tablespoon of honey or maple syrup (optional, for added sweetness)
- 1 cup of ice cubes

Instructions:
1. Gather all your ingredients and wash the fruits if necessary.
2. Chop the fruits into smaller pieces for easier blending.
3. Add the fruits, liquid, sweetener (if using), and ice cubes to a blender.
4. Blend everything together on high speed until smooth and creamy. If the consistency is too thick, you can add more liquid.
5. Taste the smoothie and adjust the sweetness or thickness according to your preference.
6. Pour the smoothie into a glass or a portable bottle.
7. You can garnish the smoothie with a sprinkle of chia seeds, shredded coconut, or 

And compare the response to the same prompt but a new settings instance while all other chat parameters are the same.

In [10]:
blank_settings = ChatRequestSettings(max_tokens=516, temperature=0.5, top_p=0.8)

chat_messages = list()
chat_messages.append(("user", user_request))
answer = await openai_chat_completion.complete_chat_async(chat_messages, blank_settings)
chat_messages.append(("assistant", str(answer)))

In [11]:
print_chat(chat_messages)

User:> I want to make a smoothie :)

ChefBot:> Great! Making a smoothie is a fun and easy way to enjoy a nutritious and delicious drink. Here's a simple recipe to get you started:

Ingredients:
- 1 ripe banana
- 1 cup of frozen berries (such as strawberries, blueberries, or mixed berries)
- 1 cup of milk (dairy or plant-based)
- 1 tablespoon of honey or maple syrup (optional, for added sweetness)
- 1/2 cup of yogurt (optional, for a creamier texture)
- Ice cubes (optional, for a colder smoothie)

Instructions:
1. Peel the banana and break it into smaller chunks.
2. Place the banana chunks, frozen berries, milk, honey or maple syrup, and yogurt (if using) into a blender.
3. Blend all the ingredients together until smooth and creamy. If the mixture seems too thick, you can add a little more milk.
4. If you prefer a colder smoothie, you can add a few ice cubes and blend again.
5. Taste the smoothie and adjust the sweetness or thickness to your liking by adding more honey, milk, or yogurt 

---

Recognize that the LLM's outputs will always have a non-deterministic aspect of it, and even with the greatest logit bias definitions, there may be banned tokens present in the output.

Prompt engineering is a tricky practice, and logit bias is just one tool to help.